In [23]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import os

def generate_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()

    return private_key, public_key

def encrypt_data(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def decrypt_data(private_key, ciphertext):
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext.decode()

def save_key_to_pem(key, key_type, filename, password=None):
    key_bytes = key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.BestAvailableEncryption(password) if password else serialization.NoEncryption()
    ) if key_type == 'private' else key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    with open(filename, 'wb') as f:
        f.write(key_bytes)
        
def read_private_key_from_pem(file_path, password=None):
    with open(file_path, 'rb') as f:
        private_key = serialization.load_pem_private_key(
            f.read(),
            password=password,
            backend=default_backend()
        )
    return private_key

def read_public_key_from_pem(file_path):
    with open(file_path, 'rb') as f:
        public_key = serialization.load_pem_public_key(
            f.read(),
            backend=default_backend()
        )
    return public_key


In [34]:
# Ejemplo de uso:
private_key, public_key = generate_key_pair()
private_key_path = 'private_key.pem'
public_key_path = 'public_key.pem'

# Almacenar las claves de forma segura en archivos PEM
save_key_to_pem(private_key, 'private', private_key_path, password=b'mypassword')
save_key_to_pem(public_key, 'public', public_key_path)

# Encriptar y desencriptar datos
message = "Hola, este es un mensaje secreto."

ciphertext = encrypt_data(public_key, message)
print("Mensaje cifrado:", ciphertext)

del private_key, public_key

private_key = read_private_key_from_pem(private_key_path, password=b'mypassword')
public_key = read_public_key_from_pem(public_key_path)

print("public_key:", public_key)

decrypted_message = decrypt_data(private_key, ciphertext)
print("Mensaje descifrado:", decrypted_message)



Mensaje cifrado: b'Fn\x8cRrT$cl>\x94\xf0\xb78\x0e\xe3\x1b\xfc\x96\x08z\x9b\x056\xf0\x89\xc4\xf2f\x1c{\xa9Q\xad\x8e\xb0\xdc\x8ek\xf3\x0f\x9a#\xbe\x08\x04\xa8\xce\xe6\x99\x8a\xa3\xb7g\xcb\xe4&\xcfU3\xcb\xf5\'\xc38\xb2a\xbe5\xe8]\x03\xb5\x90_\xe0\xa3\x13\\\xab\xcftso\x0c\x1b\x81\xea-\n&\xa1\x08\xb2\x19E\xd2\x06C\x8c\xb4\x13U\xac\xde\x18\x82\xb8\xd3!\x8c9gzF\xf8p\xe8\xacT\x04C\x86y\xc4\x12\'\xef\xb3\xa6\x8c\xef\\\x8d*l\x0ez\xac:\xcb\xe6t\x8a\xc0Tst\xdb\xfeO\xb2j\x8e`\xf9\nD\xda\x85\xd7\xcf\x81\xd84a\xa5\xcf\x9c\xc2\xdaTt\xa0y\xb1\x81*\x819\x1awJ\x05$&\'4\xfe>\\\xb0"C\x8c\x19\xd8)T\xfd\xe0\xed\x19\x9b\x18^u\xd7\xf9\x9f\xfe\xfb\xd0fy\xa1\xcd\x84l\x1el\x88<\xe0\x7f\xb1G\xd8j\xe9N\x13\xea\x93F\xc4\xe9\xa9\xf9\x80\xccF\xbb\x9dc\xb2\xfe\x1d&i\r=\xc3\xe5\xbe_'
public_key: <cryptography.hazmat.backends.openssl.rsa._RSAPublicKey object at 0x7f366530d730>
Mensaje descifrado: Hola, este es un mensaje secreto.


In [22]:
# Ejemplo de uso:
private_key, public_key = generate_key_pair()

# Almacenar las claves de forma segura en archivos PEM
save_key_to_pem(private_key, 'private', 'private_key.pem')
save_key_to_pem(public_key, 'public', 'public_key.pem')

# Encriptar y desencriptar datos
message = "Hola, este es un mensaje secreto."

ciphertext = encrypt_data(public_key, message)
print("Mensaje cifrado:", ciphertext)

decrypted_message = decrypt_data(private_key, ciphertext)
print("Mensaje descifrado:", decrypted_message)

Mensaje cifrado: b'\xc3,\xc8\xe9\xd4.D5\xc5\x0b\x9e)o\xa5\xf8%Tb\xc8\xe7\x06I\xc6\xd7\xea\x84?{\rF\x0cs\xdc*\x0f\xe0>\xd0\xf6\xb81.B\x85\xd1N\x82\x01\xfcF\xb3\x91~f\x94\x8b\x03\xad\xd8B\xd5S\x83\xc8\x9c\xc2\xcdn\xe3\xaf\x97\x89\x08\xdf\x8c\xa5\x01\xb0\x8eZx\xec\x0bT\xfda\xdc\x91o,&\xabo\xc5HT\xb7\x9a2Cw%\xeeG\xfc#iCr\xa1\xdd\xcb\x00\x8b\xc7`j\xcb\x0b56\xc0\xe2\xb8\xc5\xee\xcf\x84\xb7L\x05\x8a=\xb7\xab4\r\xd5\x88\xa2\x82]h\xd8\x9c\xa6\xbb\xc2\x16\xb1\xd6~\x87/=\x02\xa7M\x9c\xbf\x0c\x14?\x9ch\x85\xa7\x95\x95Tn\xa0U\x8e\x1d\x8b\x9e\x13\xd0j\x16\xea$x\x13\xe2o@\xfd\xcc\x00\xa4\xcai\x00T\x00j\xe8\xf9`e9\xb9\xa8\xac\xa2(\xec#Y\xc6\xc4\xc7\xe8\xc2O\xac\xd2$`s2\xa0\xb5\xc4B\xf2\xbfz\xfa\xc1\xd5\xc4p\t\xe2\x1f\xfaJ\xa1\x8bTq\xc8\xafr#\xbd\xa2\x0b\xccI\xa4\x08\xb4'
Mensaje descifrado: Hola, este es un mensaje secreto.
